# Olá, seja muito bem-vindo ao desafio de Engenheiro(a) de dados da [A3Data]

<p align="center">
  <a href="" rel="noopener">
 <img src="https://cdn-content.talentbrand.com.br/companies/394/header/ec1d053357fc5889b2daf50460552dd67b399a89.png?1606219795" alt="Project logo"></a>
</p>

### Cria um mount point no seu Google Drive para acessar os dados do DataLake

- Execute e passe o código solicitado

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### Importação das bibliotecas necessárias para realização do exercício

In [2]:
import pandas as pd
import json
import os
import glob
from sqlalchemy import create_engine, inspect, text
from pandas import DataFrame

In [3]:
engine = create_engine('sqlite://', echo=False)

In [4]:
users_path = '/content/drive/MyDrive/datalake/landing-zone/user'

df_user = pd.DataFrame()

for filename in os.listdir(users_path):
  if filename.endswith('json'):
    temp_df = pd.read_json(os.path.join(users_path,filename))
    df_user = pd.concat([df_user,temp_df],ignore_index=True)

columns_to_keep = ['user_id','first_name','last_name','email','date_of_birth']

df_user = df_user[columns_to_keep]
df_user.to_sql('user', engine, index=False)

800

In [5]:
ecommerce_path = '/content/drive/MyDrive/datalake/landing-zone/commerce'

df_ecommerce = pd.DataFrame()

for filename in os.listdir(ecommerce_path):
  if filename.endswith('json'):
    temp_df = pd.read_json(os.path.join(ecommerce_path,filename))
    df_ecommerce = pd.concat([df_ecommerce,temp_df],ignore_index=True)

columns_to_keep = ['user_id','department','product_name','price']

df_ecommerce = df_ecommerce[columns_to_keep]
df_ecommerce.to_sql('ecommerce', engine, index=False)

800

In [6]:
device_path = '/content/drive/MyDrive/datalake/landing-zone/device'

df_device = pd.DataFrame()

for filename in os.listdir(device_path):
  if filename.endswith('json'):
    temp_df = pd.read_json(os.path.join(device_path,filename))
    df_device = pd.concat([df_device,temp_df],ignore_index=True)

columns_to_keep = ['user_id','platform','model']

df_device = df_device[columns_to_keep]
df_device.to_sql('device', engine, index=False)

800

In [9]:
connection = engine.connect()

query = text("""
SELECT platform, count(*)
FROM user
JOIN device ON user.user_id = device.user_id
JOIN ecommerce ON user.user_id = ecommerce.user_id
GROUP BY platform
""")

result = connection.execute(query).fetchall()

connection.close()

platform_count = pd.DataFrame(result)
platform_count

,platform,count(*)
0,Danger OS,1
1,Windows 10 Mobile,3
2,Windows Phone,1
3,Windows RT,1


In [10]:
platform_count.to_csv('/content/drive/MyDrive/datalake/consumer-zone/plataform_count.csv', index=False)